In [1]:
from Utils.DataProvider import mysql_config, api_key_sql
import mysql.connector
import requests
import pandas as pd
import datetime
import io

#TO-DO: 
# 1. Testing each function
# 2. For return type as csv, test our function to make sure it will not error out.

class AlphaVantage(object):

    def __init__(self) -> None:
        cnx = mysql.connector.connect(**mysql_config)
        cursor = cnx.cursor()
        cursor.execute(api_key_sql, [AlphaVantage.__name__])

        for _ in cursor:
            self.api_key = _[0]

        try: 
            url = 'https://www.alphavantage.co/documentation/'
            request = requests.get(url, timeout=5)
            print('AlphaVantage Connection Established')
        except:
            print("AlphaVantage Connection Failed")

    def _alphaVantage_api_call(self, fn, pandas_return=False,**kvarg):
        url = 'https://www.alphavantage.co/query?function={functionName}'.format(functionName=fn)

        for key in kvarg.keys():
            if kvarg[key] is None: continue
            url += '&{key}={value}'.format(key=key, value=kvarg[key])
        
        url += '&{key}={value}'.format(key='apikey', value=self.api_key)
        
        if pandas_return:
            r = requests.get(url)
            data = pd.read_csv(io.StringIO(r.content.decode('utf-8')))
        else:
            r = requests.get(url)
            data = r.json()

        return data

######################################################################################################
#################################  Fundamental Data   ################################################
######################################################################################################

    def CompanyOverview(self, symbol):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=OVERVIEW
        ❚ Required: symbol
            The symbol of the token of your choice. For example: symbol=IBM.
        '''

        return self._alphaVantage_api_call('OVERVIEW', False, symbol=symbol)

    def IncomeStatement(self, symbol):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=INCOME_STATEMENT
        ❚ Required: symbol
            The symbol of the token of your choice. For example: symbol=IBM.
        '''

        return self._alphaVantage_api_call('INCOME_STATEMENT', pandas_return=False, symbol=symbol)

    def BalanceSheet(self, symbol):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=BALANCE_SHEET
        ❚ Required: symbol
            The symbol of the token of your choice. For example: symbol=IBM.
        '''

        return self._alphaVantage_api_call('BALANCE_SHEET', False, symbol=symbol)
    
    def CashflowStatement(self, symbol):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=CASH_FLOW
        ❚ Required: symbol
            The symbol of the token of your choice. For example: symbol=IBM.
        '''

        return self._alphaVantage_api_call('CASH_FLOW', False, symbol=symbol)
    
    def ReportedEPS_TimeSeries(self, symbol):
        '''
        ❚ Required: function
            The function of your choice. In this case, function=EARNINGS
        ❚ Required: symbol
            The symbol of the token of your choice. For example: symbol=IBM.
        '''
        
        return self._alphaVantage_api_call('EARNINGS', False, symbol=symbol)
    
    def ListingStatus(self, date=datetime.date.today(), state='active'):
        '''
        ❚ Required: function
            The API function of your choice. In this case, function=LISTING_STATUS
        ❚ Optional: date
            If no date is set, the API endpoint will return a list of active or delisted symbols as of the latest trading day. 
            If a date is set, the API endpoint will "travel back" in time and return a list of active or delisted symbols on that particular date in history. 
            Any YYYY-MM-DD date later than 2010-01-01 is supported. For example, date=2013-08-03
        ❚ Optional: state
            By default, state=active and the API will return a list of actively traded stocks and ETFs. Set state=delisted to query a list of delisted assets.
        '''
        
        return self._alphaVantage_api_call('LISTING_STATUS', True, date=date, state=state)

    def EarningsCalendar(self, symbol = None, horizon = '3month'):
        '''
        API Parameters
        ❚ Required: function
            The API function of your choice. In this case, function=EARNINGS_CALENDAR
        ❚ Optional: symbol
            By default, no symbol will be set for this API. 
            When no symbol is set, the API endpoint will return the full list of company earnings scheduled. 
            If a symbol is set, the API endpoint will return the expected earnings for that specific symbol. For example, symbol=IBM
        ❚ Optional: horizon
            By default, horizon=3month and the API will return a list of expected company earnings in the next 3 months. 
            You may set horizon=6month or horizon=12month to query the earnings scheduled for the next 6 months or 12 months, respectively.
        '''
        
        return self._alphaVantage_api_call('EARNINGS_CALENDAR', True, symbol=symbol, horizon=horizon)
    
    def IPOCalendar(self):
        '''
        API Parameters
        ❚ Required: function
            The API function of your choice. In this case, function=IPO_CALENDAR
        '''
        
        return self._alphaVantage_api_call('IPO_CALENDAR', True)

######################################################################################################
#################################   Core Stock APIs   ################################################
######################################################################################################

    def Time_Series_Intraday(self, symbol, interval, adjusted='true', outputsize='compact', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_INTRADAY
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Required: interval
            Time interval between two consecutive data points in the time series. The following values are supported: 1min, 5min, 15min, 30min, 60min
        ❚ Optional: adjusted
            By default, adjusted=true and the output time series is adjusted by historical split and dividend events. Set adjusted=false to query raw (as-traded) intraday values.
        ❚ Optional: outputsize
            By default, outputsize=compact. 
            Strings compact and full are accepted with the following specifications: 
            compact returns only the latest 100 data points in the intraday time series; full returns the full-length intraday time series. 
            The "compact" option is recommended if you would like to reduce the data size of each API call.
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the intraday time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''
        
        return self._alphaVantage_api_call('TIME_SERIES_INTRADAY', datatype=='csv', symbol=symbol, interval=interval, adjusted=adjusted, outputsize=outputsize, datatype=datatype)

    def Time_Serires_Intraday_Extended(self, symbol, interval, slice='year1month1', adjusted='true'):
        '''
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_INTRADAY_EXTENDED
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Required: interval
            Time interval between two consecutive data points in the time series. The following values are supported: 1min, 5min, 15min, 30min, 60min
        ❚ Required: slice
            Two years of minute-level intraday data contains over 2 million data points, which can take up to Gigabytes of memory. 
            To ensure optimal API response speed, the trailing 2 years of intraday data is evenly divided into 24 "slices" 
            - year1month1, year1month2, year1month3, ..., year1month11, year1month12, year2month1, year2month2, year2month3, ..., year2month11, year2month12. 
            Each slice is a 30-day window, with year1month1 being the most recent and year2month12 being the farthest from today. 
            By default, slice=year1month1.
        ❚ Optional: adjusted
            By default, adjusted=true and the output time series is adjusted by historical split and dividend events. Set adjusted=false to query raw (as-traded) intraday values.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_INTRADAY_EXTENDED', True, symbol=symbol, interval=interval, slice=slice, adjusted=adjusted)

    def Time_Serires_Daily(self, symbol, outputsize='compact', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_DAILY
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: outputsize
            By default, outputsize=compact. 
            Strings compact and full are accepted with the following specifications: 
            compact returns only the latest 100 data points; full returns the full-length time series of 20+ years of historical data. 
            The "compact" option is recommended if you would like to reduce the data size of each API call.
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the daily time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_DAILY', datatype=='csv', symbol=symbol, outputsize=outputsize, datatype=datatype)

    def Time_Serires_Daily_Adjusted(self, symbol, outputsize='compact', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
        The time series of your choice. In this case, function=TIME_SERIES_DAILY_ADJUSTED
        ❚ Required: symbol
        The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: outputsize
            By default, outputsize=compact. 
            Strings compact and full are accepted with the following specifications: 
            compact returns only the latest 100 data points; full returns the full-length time series of 20+ years of historical data. 
            The "compact" option is recommended if you would like to reduce the data size of each API call.
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the daily time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_DAILY_ADJUSTED', datatype=='csv', symbol=symbol, outputsize=outputsize, datatype=datatype)

    def Time_Serires_Weekly(self, symbol, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_WEEKLY
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the weekly time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_WEEKLY', datatype=='csv', symbol=symbol, datatype=datatype)
    
    def Time_Serires_Weekly_Adjusted(self, symbol, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_WEEKLY_ADJUSTED
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the weekly time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_WEEKLY_ADJUSTED', datatype=='csv', symbol=symbol, datatype=datatype)

    def Time_Serires_Monthly(self, symbol, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_MONTHLY
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the weekly time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_MONTHLY', datatype=='csv', symbol=symbol, datatype=datatype)
    
    def Time_Serires_Monthly_Adjusted(self, symbol, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The time series of your choice. In this case, function=TIME_SERIES_MONTHLY_ADJUSTED
        ❚ Required: symbol
            The name of the equity of your choice. For example: symbol=IBM
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the weekly time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TIME_SERIES_MONTHLY_ADJUSTED', datatype=='csv', symbol=symbol, datatype=datatype)
    
    def Quote_Endpoint(self, symbol, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The API function of your choice.
        ❚ Required: symbol
            The symbol of the global token of your choice. For example: symbol=IBM.
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the quote data in JSON format; csv returns the quote data as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('GLOBAL_QUOTE', datatype=='csv', symbol=symbol, datatype=datatype)
    
    def Search_Endpoint(self, keywords, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The API function of your choice. In this case, function=SYMBOL_SEARCH
        ❚ Required: keywords
            A text string of your choice. For example: keywords=microsoft.
        ❚ Optional: datatype
            By default, datatype=json. 
            Strings json and csv are accepted with the following specifications: 
            json returns the search results in JSON format; csv returns the search results as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('SYMBOL_SEARCH', datatype=='csv', keywords=keywords, datatype=datatype)

    def Market_Status(self):
        '''
        API Parameters
        ❚ Required: function
            The API function of your choice. In this case, function=MARKET_STATUS
        '''

        return self._alphaVantage_api_call('MARKET_STATUS')    

######################################################################################################
#################################   Alpha Intelligence   #############################################
######################################################################################################

    def Market_News_Sentiment(self, tickers, topics, time_from, time_to, sort='LATEST', limit='50'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=NEWS_SENTIMENT
        ❚ Optional: tickers
            The stock/crypto/forex symbols of your choice. For example: tickers=IBM will filter for articles that mention the IBM ticker; 
            tickers=COIN,CRYPTO:BTC,FOREX:USD will filter for articles that simultaneously mention Coinbase (COIN), Bitcoin (CRYPTO:BTC), 
            and US Dollar (FOREX:USD) in their content.
        ❚ Optional: topics
            The news topics of your choice. For example: topics=technology will filter for articles that write about the technology sector; 
            topics=technology,ipo will filter for articles that simultaneously cover technology and IPO in their content. 
            Below is the full list of supported topics:
                Blockchain: blockchain
                Earnings: earnings
                IPO: ipo
                Mergers & Acquisitions: mergers_and_acquisitions
                Financial Markets: financial_markets
                Economy - Fiscal Policy (e.g., tax reform, government spending): economy_fiscal
                Economy - Monetary Policy (e.g., interest rates, inflation): economy_monetary
                Economy - Macro/Overall: economy_macro
                Energy & Transportation: energy_transportation
                Finance: finance
                Life Sciences: life_sciences
                Manufacturing: manufacturing
                Real Estate & Construction: real_estate
                Retail & Wholesale: retail_wholesale
                Technology: technology
        ❚ Optional: time_from and time_to
           The time range of the news articles you are targeting, in YYYYMMDDTHHMM format. 
           For example: time_from=20220410T0130. 
           If time_from is specified but time_to is missing, the API will return articles published between the time_from value and the current time.
        ❚ Optional: sort
            By default, sort=LATEST and the API will return the latest articles first. You can also set sort=EARLIEST or sort=RELEVANCE based on your use case.
        ❚ Optional: limit
            By default, limit=50 and the API will return up to 50 matching results. 
            You can also set limit=200 to output up to 200 results. 
            If you are looking for an even higher output limit, please contact support@alphavantage.co to have your limit boosted.
        '''

        return self._alphaVantage_api_call('NEWS_SENTIMENT', False, tickers=tickers, topics=topics, time_from=time_from, time_to=time_to, sort=sort, limit=limit)
    
######################################################################################################
#################################  Economic Indicators  ##############################################
######################################################################################################

    def Real_GDP(self, interval='annual', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=REAL_GDP
        ❚ Optional: interval
            By default, interval=annual. Strings quarterly and annual are accepted.
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('REAL_GDP', datatype=='csv', interval=interval, datatype=datatype)
  
    def Real_GDP_per_Capita(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=REAL_GDP_PER_CAPITA
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('REAL_GDP_PER_CAPITA', datatype=='csv', datatype=datatype)

    def Treasury_Yield(self, interval='monthly', maturity='10year', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=TREASURY_YIELD
        ❚ Optional: interval
            By default, interval=monthly. Strings daily, weekly, and monthly are accepted.
        ❚ Optional: maturity
            By default, maturity=10year. Strings 3month, 2year, 5year, 7year, 10year, and 30year are accepted.
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('TREASURY_YIELD', datatype=='csv',  interval=interval, maturity=maturity, datatype=datatype)
    
    def Federal_Funds_Rate(self, interval='monthly', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=FEDERAL_FUNDS_RATE
        ❚ Optional: interval
            By default, interval=monthly. Strings daily, weekly, and monthly are accepted.
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('FEDERAL_FUNDS_RATE', datatype=='csv',  interval=interval, datatype=datatype)
    
    def CPI(self, interval='monthly', datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=CPI
        ❚ Optional: interval
            By default, interval=monthly. Strings monthly and semiannual are accepted.
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('CPI', datatype=='csv',  interval=interval, datatype=datatype)
    
    def Inflation(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=INFLATION
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('INFLATION', datatype=='csv',  datatype=datatype)
    
    def Retail_Sales(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=RETAIL_SALES
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('RETAIL_SALES', datatype=='csv',  datatype=datatype)
    
    def Durables(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=DURABLES
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('DURABLES', datatype=='csv',  datatype=datatype)
    
    def Unemployment(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=UNEMPLOYMENT
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('UNEMPLOYMENT', datatype=='csv',  datatype=datatype)
    
    def Nonfarm_Payroll(self, datatype='json'):
        '''
        API Parameters
        ❚ Required: function
            The function of your choice. In this case, function=NONFARM_PAYROLL
        ❚ Optional: datatype
            By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the time series in JSON format; csv returns the time series as a CSV (comma separated value) file.
        '''

        return self._alphaVantage_api_call('NONFARM_PAYROLL', datatype=='csv',  datatype=datatype)

In [2]:
av = AlphaVantage()

AlphaVantage Connection Established


In [3]:
symbol = 'AAPL'

In [5]:
CompanyOverview = av.CompanyOverview(symbol)
CompanyOverview

{'Symbol': 'AAPL',
 'AssetType': 'Common Stock',
 'Name': 'Apple Inc',
 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
 'CIK': '320193',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2022-12-31',
 'MarketCapitalization': '2494330000000',
 'EBITDA': '125287997000',
 'PERatio': '25.79',
 'PEGRatio': '2.75',
 'BookVal

In [6]:
IncomeStatement = av.IncomeStatement(symbol)
IncomeStatement

{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2022-09-30',
   'reportedCurrency': 'USD',
   'grossProfit': '170782000000',
   'totalRevenue': '391397000000',
   'costOfRevenue': '248640000000',
   'costofGoodsAndServicesSold': '223546000000',
   'operatingIncome': '119437000000',
   'sellingGeneralAndAdministrative': '25094000000',
   'researchAndDevelopment': '26251000000',
   'operatingExpenses': '51345000000',
   'investmentIncomeNet': '2825000000',
   'netInterestIncome': '-2931000000',
   'interestIncome': '106000000',
   'interestExpense': '2931000000',
   'nonInterestIncome': '394328000000',
   'otherNonOperatingIncome': '-228000000',
   'depreciation': '8700000000',
   'depreciationAndAmortization': '11104000000',
   'incomeBeforeTax': '119103000000',
   'incomeTaxExpense': '19300000000',
   'interestAndDebtExpense': '2931000000',
   'netIncomeFromContinuingOperations': '99803000000',
   'comprehensiveIncomeNetOfTax': '88531000000',
   'ebit': '122034000000',
   '

In [7]:
BalanceSheet = av.BalanceSheet(symbol)
BalanceSheet

{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2022-09-30',
   'reportedCurrency': 'USD',
   'totalAssets': '352755000000',
   'totalCurrentAssets': '135405000000',
   'cashAndCashEquivalentsAtCarryingValue': '23646000000',
   'cashAndShortTermInvestments': '48304000000',
   'inventory': '4946000000',
   'currentNetReceivables': '60932000000',
   'totalNonCurrentAssets': '217350000000',
   'propertyPlantEquipment': '42117000000',
   'accumulatedDepreciationAmortizationPPE': '72340000000',
   'intangibleAssets': 'None',
   'intangibleAssetsExcludingGoodwill': 'None',
   'goodwill': 'None',
   'investments': '292870000000',
   'longTermInvestments': '120805000000',
   'shortTermInvestments': '24658000000',
   'otherCurrentAssets': '21223000000',
   'otherNonCurrentAssets': '54428000000',
   'totalLiabilities': '302083000000',
   'totalCurrentLiabilities': '153982000000',
   'currentAccountsPayable': '64115000000',
   'deferredRevenue': '20312000000',
   'currentDebt': '21239

In [8]:
CashflowStatement = av.CashflowStatement(symbol)
CashflowStatement

{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2022-09-30',
   'reportedCurrency': 'USD',
   'operatingCashflow': '122151000000',
   'paymentsForOperatingActivities': '4665000000',
   'proceedsFromOperatingActivities': 'None',
   'changeInOperatingLiabilities': '15558000000',
   'changeInOperatingAssets': '14358000000',
   'depreciationDepletionAndAmortization': '11104000000',
   'capitalExpenditures': '10708000000',
   'changeInReceivables': '9343000000',
   'changeInInventory': '-1484000000',
   'profitLoss': '99803000000',
   'cashflowFromInvestment': '-22354000000',
   'cashflowFromFinancing': '-110749000000',
   'proceedsFromRepaymentsOfShortTermDebt': '7910000000',
   'paymentsForRepurchaseOfCommonStock': '89402000000',
   'paymentsForRepurchaseOfEquity': '89402000000',
   'paymentsForRepurchaseOfPreferredStock': 'None',
   'dividendPayout': '14841000000',
   'dividendPayoutCommonStock': '14841000000',
   'dividendPayoutPreferredStock': 'None',
   'proceedsFromIssuan

In [10]:
ReportedEPS_TimeSeries = av.ReportedEPS_TimeSeries(symbol)
ReportedEPS_TimeSeries

{'symbol': 'AAPL',
 'annualEarnings': [{'fiscalDateEnding': '2022-12-31', 'reportedEPS': '1.88'},
  {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'},
  {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'},
  {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'},
  {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'},
  {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'},
  {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'},
  {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'},
  {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'},
  {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'},
  {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'},
  {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'},
  {'fiscalDateEnding': '2009-09-30', 'reportedEPS': '0.2225'},
  {'fiscalDateEnding': '2008-09-30', 'r

In [11]:
ListingStatus = av.ListingStatus()
ListingStatus

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,NaN,Active
...,...,...,...,...,...,...,...
11677,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active
11678,ZYRX,Global Earnings Capital Ltd,NASDAQ,Stock,2007-07-13,NaN,Active
11679,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,NaN,Active
11680,ZZK,NaN,NYSE ARCA,Stock,2020-07-22,NaN,Active


In [12]:
ListingStatus = av.ListingStatus('2023-03-01', state='delisted')
ListingStatus

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,AA-W,Alcoa Corporation When Issued,NYSE,Stock,2016-10-18,2016-11-08,Delisted
1,AAAP,Advanced Accelerator Applications SA,NASDAQ,Stock,2015-11-11,2018-02-20,Delisted
2,AABA,Altaba Inc,NASDAQ,Stock,1996-04-12,2019-11-06,Delisted
3,AAC,American Addiction Centers,NYSE,Stock,2014-10-02,2021-04-19,Delisted
4,AACC,Asset Acceptance Capital Corp,NASDAQ,Stock,2004-02-05,2013-10-17,Delisted
...,...,...,...,...,...,...,...
7105,ZWRK,Z-Work Acquisition Corp - Class A,NASDAQ,Stock,2021-04-05,2022-12-08,Delisted
7106,ZWRKU,Z-Work Acquisition Corp - Units (1 Ord Class A...,NASDAQ,Stock,2021-01-29,2022-12-08,Delisted
7107,ZWRKW,Z-Work Acquisition Corp - Warrants (29/01/2026),NASDAQ,Stock,2021-03-24,2022-12-08,Delisted
7108,ZX,China Zenix Auto International Ltd,NYSE,Stock,2011-05-16,2018-06-14,Delisted


In [13]:
EarningsCalendar = av.EarningsCalendar(symbol)
EarningsCalendar

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,AAPL,Apple Inc,2023-04-26,2023-03-31,1.43,USD


In [14]:
EarningsCalendar = av.EarningsCalendar(horizon='6month')
EarningsCalendar

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,A,Agilent Technologies Inc,2023-05-22,2023-04-30,1.2600,USD
1,A,Agilent Technologies Inc,2023-08-14,2023-07-31,NaN,USD
2,AA,Alcoa Corp,2023-04-18,2023-03-31,0.1300,USD
3,AA,Alcoa Corp,2023-07-18,2023-06-30,NaN,USD
4,AABVF,AABVF,2023-05-01,2023-04-30,NaN,USD
...,...,...,...,...,...,...
13107,ZYME,Zymeworks BC Inc,2023-08-02,2023-06-30,NaN,USD
13108,ZYNE,Zynerba Pharmaceuticals Inc,2023-05-15,2023-03-31,-0.1867,USD
13109,ZYNE,Zynerba Pharmaceuticals Inc,2023-08-08,2023-06-30,NaN,USD
13110,ZYXI,Zynex Inc,2023-04-26,2023-03-31,0.0100,USD


In [15]:
IPOCalendar = av.IPOCalendar()
IPOCalendar

,symbol,name,ipoDate,priceRangeLow,priceRangeHigh,currency,exchange
0,ETEC,iShares Breakthrough Environmental Solutions ETF,2023-03-30,0,0,USD,NASDAQ
1,DISTR,Distoken Acquisition Corporation Right,2023-03-30,0,0,USD,NASDAQ
2,DIST,Distoken Acquisition Corporation Ordinary Shares,2023-03-30,0,0,USD,NASDAQ
3,CHSN,Chanson International Holding Class A Ordinary...,2023-03-30,0,0,USD,NASDAQ
4,CYTOW,Altamira Therapeutics Ltd. Warrant,2023-03-30,0,0,USD,NASDAQ
5,DISTW,Distoken Acquisition Corporation Warrant,2023-03-30,0,0,USD,NASDAQ
6,TCJH,Top KingWin Ltd Class A Ordinary Shares,2023-03-30,0,0,USD,NASDAQ
7,MGIH,Millennium Group International Holdings Limite...,2023-03-30,0,0,USD,NASDAQ
8,APLMW,Apollomics Inc. Warrant,2023-03-30,0,0,USD,NASDAQ
9,APLM,Apollomics Inc. Class A Ordinary Shares,2023-03-30,0,0,USD,NASDAQ


In [16]:
interval='60min'
Time_Series_Intraday = av.Time_Series_Intraday(symbol, interval=interval, adjusted='true', outputsize='compact', datatype='json')
Time_Series_Intraday

{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-29 20:00:00',
  '4. Interval': '60min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (60min)': {'2023-03-29 20:00:00': {'1. open': '160.5999',
   '2. high': '160.6000',
   '3. low': '160.2100',
   '4. close': '160.3500',
   '5. volume': '26612'},
  '2023-03-29 19:00:00': {'1. open': '160.6100',
   '2. high': '160.6500',
   '3. low': '160.5100',
   '4. close': '160.5900',
   '5. volume': '20516'},
  '2023-03-29 18:00:00': {'1. open': '160.6400',
   '2. high': '160.7700',
   '3. low': '160.6000',
   '4. close': '160.6102',
   '5. volume': '217606'},
  '2023-03-29 17:00:00': {'1. open': '160.7800',
   '2. high': '160.9400',
   '3. low': '160.5700',
   '4. close': '160.6300',
   '5. volume': '1988778'},
  '2023-03-29 16:00:00': {'1. open': '160.4500',
   '2. high': '161.0500',
   '3. low': '160.2950',
   '4. cl

In [17]:
interval='60min'
Time_Series_Intraday = av.Time_Series_Intraday(symbol, interval=interval, adjusted='true', outputsize='compact', datatype='csv')
Time_Series_Intraday

,timestamp,open,high,low,close,volume
0,2023-03-29 20:00:00,160.5999,160.60,160.210,160.3500,26612
1,2023-03-29 19:00:00,160.6100,160.65,160.510,160.5900,20516
2,2023-03-29 18:00:00,160.6400,160.77,160.600,160.6102,217606
3,2023-03-29 17:00:00,160.7800,160.94,160.570,160.6300,1988778
4,2023-03-29 16:00:00,160.4500,161.05,160.295,160.7800,9628869
...,...,...,...,...,...,...
95,2023-03-22 05:00:00,158.0000,158.85,158.000,158.2100,27033
96,2023-03-21 20:00:00,159.0500,159.27,158.940,158.9700,47194
97,2023-03-21 19:00:00,158.9500,159.05,158.890,159.0500,20489
98,2023-03-21 18:00:00,159.1900,159.21,158.860,158.8600,61095


In [18]:
Time_Serires_Intraday_Extended = av.Time_Serires_Intraday_Extended(symbol, interval, slice='year1month1', adjusted='true')
Time_Serires_Intraday_Extended

,time,open,high,low,close,volume
0,2023-03-29 20:00:00,160.5999,160.60,160.210,160.3500,26612
1,2023-03-29 19:00:00,160.6100,160.65,160.510,160.5900,20516
2,2023-03-29 18:00:00,160.6400,160.77,160.600,160.6102,217606
3,2023-03-29 17:00:00,160.7800,160.94,160.570,160.6300,1988778
4,2023-03-29 16:00:00,160.4500,161.05,160.295,160.7800,9628869
...,...,...,...,...,...,...
347,2023-02-28 09:00:00,147.8901,148.26,147.280,147.5900,340826
348,2023-02-28 08:00:00,147.8300,148.23,147.750,148.1100,60377
349,2023-02-28 07:00:00,147.6300,147.85,147.460,147.8300,17702
350,2023-02-28 06:00:00,147.2000,147.69,147.180,147.6100,18876


In [20]:
Time_Serires_Daily = av.Time_Serires_Daily(symbol, outputsize='compact', datatype='json')
Time_Serires_Daily

{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}

In [21]:
Time_Serires_Daily_Adjusted = av.Time_Serires_Daily_Adjusted(symbol, outputsize='compact', datatype='json')
Time_Serires_Daily_Adjusted

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-30',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-03-30': {'1. open': '161.53',
   '2. high': '162.47',
   '3. low': '161.271',
   '4. close': '162.36',
   '5. adjusted close': '162.36',
   '6. volume': '46805370',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-03-29': {'1. open': '159.37',
   '2. high': '161.05',
   '3. low': '159.35',
   '4. close': '160.77',
   '5. adjusted close': '160.77',
   '6. volume': '51305691',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-03-28': {'1. open': '157.97',
   '2. high': '158.49',
   '3. low': '155.98',
   '4. close': '157.65',
   '5. adjusted close': '157.65',
   '6. volume': '45992152',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-03-27': {'1. open': '159.94',
  

In [22]:
Time_Serires_Daily_Adjusted = av.Time_Serires_Daily_Adjusted(symbol, outputsize='compact', datatype='csv')
Time_Serires_Daily_Adjusted

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-03-30,161.53,162.470,161.2710,162.36,162.360000,46805370,0.00,1.0
1,2023-03-29,159.37,161.050,159.3500,160.77,160.770000,51305691,0.00,1.0
2,2023-03-28,157.97,158.490,155.9800,157.65,157.650000,45992152,0.00,1.0
3,2023-03-27,159.94,160.770,157.8700,158.28,158.280000,52390266,0.00,1.0
4,2023-03-24,158.86,160.340,157.8500,160.25,160.250000,59256343,0.00,1.0
...,...,...,...,...,...,...,...,...,...
95,2022-11-10,141.24,146.870,139.5000,146.87,146.461486,118854028,0.00,1.0
96,2022-11-09,138.50,138.550,134.5933,134.87,134.494864,74917794,0.00,1.0
97,2022-11-08,140.41,141.430,137.4900,139.50,139.111986,89908477,0.00,1.0
98,2022-11-07,137.11,139.145,135.6710,138.92,138.533599,83374628,0.00,1.0


In [23]:
Time_Serires_Weekly = av.Time_Serires_Weekly(symbol, datatype='json')
Time_Serires_Weekly

{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-30',
  '4. Time Zone': 'US/Eastern'},
 'Weekly Time Series': {'2023-03-30': {'1. open': '159.9400',
   '2. high': '162.4700',
   '3. low': '155.9800',
   '4. close': '162.3600',
   '5. volume': '196493479'},
  '2023-03-24': {'1. open': '155.0700',
   '2. high': '162.1400',
   '3. low': '154.1500',
   '4. close': '160.2500',
   '5. volume': '350159914'},
  '2023-03-17': {'1. open': '147.8050',
   '2. high': '156.7400',
   '3. low': '147.7000',
   '4. close': '155.0000',
   '5. volume': '410519933'},
  '2023-03-10': {'1. open': '153.7850',
   '2. high': '156.3000',
   '3. low': '147.6096',
   '4. close': '148.5000',
   '5. volume': '313350829'},
  '2023-03-03': {'1. open': '147.7100',
   '2. high': '151.1100',
   '3. low': '143.9000',
   '4. close': '151.0300',
   '5. volume': '274036517'},
  '2023-02-24': {'1. open': '150.2000',
   '2. high': '151

In [24]:
Time_Serires_Weekly = av.Time_Serires_Weekly(symbol, datatype='csv')
Time_Serires_Weekly

,timestamp,open,high,low,close,volume
0,2023-03-30,159.940,162.47,155.9800,162.36,196493479
1,2023-03-24,155.070,162.14,154.1500,160.25,350159914
2,2023-03-17,147.805,156.74,147.7000,155.00,410519933
3,2023-03-10,153.785,156.30,147.6096,148.50,313350829
4,2023-03-03,147.710,151.11,143.9000,151.03,274036517
...,...,...,...,...,...,...
1216,1999-12-10,114.560,118.00,99.0000,103.00,25152800
1217,1999-12-03,94.250,115.56,93.2500,115.00,28046300
1218,1999-11-26,91.750,95.50,88.5000,95.06,9757600
1219,1999-11-19,89.620,94.75,88.0600,92.44,13711000


In [25]:
Time_Serires_Weekly_Adjusted = av.Time_Serires_Weekly_Adjusted(symbol, datatype='json')
Time_Serires_Weekly_Adjusted

{'Meta Data': {'1. Information': 'Weekly Adjusted Prices and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-30',
  '4. Time Zone': 'US/Eastern'},
 'Weekly Adjusted Time Series': {'2023-03-30': {'1. open': '159.9400',
   '2. high': '162.4700',
   '3. low': '155.9800',
   '4. close': '162.3600',
   '5. adjusted close': '162.3600',
   '6. volume': '196493479',
   '7. dividend amount': '0.0000'},
  '2023-03-24': {'1. open': '155.0700',
   '2. high': '162.1400',
   '3. low': '154.1500',
   '4. close': '160.2500',
   '5. adjusted close': '160.2500',
   '6. volume': '350159914',
   '7. dividend amount': '0.0000'},
  '2023-03-17': {'1. open': '147.8050',
   '2. high': '156.7400',
   '3. low': '147.7000',
   '4. close': '155.0000',
   '5. adjusted close': '155.0000',
   '6. volume': '410519933',
   '7. dividend amount': '0.0000'},
  '2023-03-10': {'1. open': '153.7850',
   '2. high': '156.3000',
   '3. low': '147.6096',
   '4. close': '148.5000',
   '5. adjusted close': '148.5

In [26]:
Time_Serires_Weekly_Adjusted = av.Time_Serires_Weekly_Adjusted(symbol, datatype='csv')
Time_Serires_Weekly_Adjusted

,timestamp,open,high,low,close,adjusted close,volume,dividend amount
0,2023-03-30,159.940,162.47,155.9800,162.36,162.3600,196493479,0.0
1,2023-03-24,155.070,162.14,154.1500,160.25,160.2500,350159914,0.0
2,2023-03-17,147.805,156.74,147.7000,155.00,155.0000,410519933,0.0
3,2023-03-10,153.785,156.30,147.6096,148.50,148.5000,313350829,0.0
4,2023-03-03,147.710,151.11,143.9000,151.03,151.0300,274036517,0.0
...,...,...,...,...,...,...,...,...
1216,1999-12-10,114.560,118.00,99.0000,103.00,0.7816,25152800,0.0
1217,1999-12-03,94.250,115.56,93.2500,115.00,0.8727,28046300,0.0
1218,1999-11-26,91.750,95.50,88.5000,95.06,0.7214,9757600,0.0
1219,1999-11-19,89.620,94.75,88.0600,92.44,0.7015,13711000,0.0


In [27]:
Time_Serires_Monthly = av.Time_Serires_Monthly(symbol, datatype='json')
Time_Serires_Monthly

{'Meta Data': {'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-30',
  '4. Time Zone': 'US/Eastern'},
 'Monthly Time Series': {'2023-03-30': {'1. open': '146.8300',
   '2. high': '162.4700',
   '3. low': '143.9000',
   '4. close': '162.3600',
   '5. volume': '1449015204'},
  '2023-02-28': {'1. open': '143.9700',
   '2. high': '157.3800',
   '3. low': '141.3200',
   '4. close': '147.4100',
   '5. volume': '1307294493'},
  '2023-01-31': {'1. open': '130.2800',
   '2. high': '147.2300',
   '3. low': '124.1700',
   '4. close': '144.2900',
   '5. volume': '1443652725'},
  '2022-12-30': {'1. open': '148.2100',
   '2. high': '150.9199',
   '3. low': '125.8700',
   '4. close': '129.9300',
   '5. volume': '1675731304'},
  '2022-11-30': {'1. open': '155.0800',
   '2. high': '155.4500',
   '3. low': '134.3800',
   '4. close': '148.0300',
   '5. volume': '1724948219'},
  '2022-10-31': {'1. open': '138.2100',
   '2. high

In [28]:
Time_Serires_Monthly = av.Time_Serires_Monthly(symbol, datatype='csv')
Time_Serires_Monthly

,timestamp,open,high,low,close,volume
0,2023-03-30,146.83,162.4700,143.90,162.36,1449015204
1,2023-02-28,143.97,157.3800,141.32,147.41,1307294493
2,2023-01-31,130.28,147.2300,124.17,144.29,1443652725
3,2022-12-30,148.21,150.9199,125.87,129.93,1675731304
4,2022-11-30,155.08,155.4500,134.38,148.03,1724948219
...,...,...,...,...,...,...
275,2000-04-28,135.50,139.5000,104.87,124.06,77342900
276,2000-03-31,118.56,150.3800,114.00,135.81,77663900
277,2000-02-29,104.00,119.9400,97.00,114.62,65355200
278,2000-01-31,104.87,121.5000,86.50,103.75,112099800


In [29]:
Time_Serires_Monthly_Adjusted = av.Time_Serires_Monthly_Adjusted(symbol, datatype='json')
Time_Serires_Monthly_Adjusted

{'Meta Data': {'1. Information': 'Monthly Adjusted Prices and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-03-30',
  '4. Time Zone': 'US/Eastern'},
 'Monthly Adjusted Time Series': {'2023-03-30': {'1. open': '146.8300',
   '2. high': '162.4700',
   '3. low': '143.9000',
   '4. close': '162.3600',
   '5. adjusted close': '162.3600',
   '6. volume': '1449015204',
   '7. dividend amount': '0.0000'},
  '2023-02-28': {'1. open': '143.9700',
   '2. high': '157.3800',
   '3. low': '141.3200',
   '4. close': '147.4100',
   '5. adjusted close': '147.4100',
   '6. volume': '1307294493',
   '7. dividend amount': '0.2300'},
  '2023-01-31': {'1. open': '130.2800',
   '2. high': '147.2300',
   '3. low': '124.1700',
   '4. close': '144.2900',
   '5. adjusted close': '144.0706',
   '6. volume': '1443652725',
   '7. dividend amount': '0.0000'},
  '2022-12-30': {'1. open': '148.2100',
   '2. high': '150.9199',
   '3. low': '125.8700',
   '4. close': '129.9300',
   '5. adjusted close': '

In [30]:
Time_Serires_Monthly_Adjusted = av.Time_Serires_Monthly_Adjusted(symbol, datatype='csv')
Time_Serires_Monthly_Adjusted

,timestamp,open,high,low,close,adjusted close,volume,dividend amount
0,2023-03-30,146.83,162.4700,143.90,162.36,162.3600,1449015204,0.0000
1,2023-02-28,143.97,157.3800,141.32,147.41,147.4100,1307294493,0.2300
2,2023-01-31,130.28,147.2300,124.17,144.29,144.0706,1443652725,0.0000
3,2022-12-30,148.21,150.9199,125.87,129.93,129.7324,1675731304,0.1667
4,2022-11-30,155.08,155.4500,134.38,148.03,147.6183,1724948219,0.2300
...,...,...,...,...,...,...,...,...
275,2000-04-28,135.50,139.5000,104.87,124.06,0.9415,77342900,0.0000
276,2000-03-31,118.56,150.3800,114.00,135.81,1.0306,77663900,0.0000
277,2000-02-29,104.00,119.9400,97.00,114.62,0.8698,65355200,0.0000
278,2000-01-31,104.87,121.5000,86.50,103.75,0.7873,112099800,0.0000


In [31]:
Quote_Endpoint = av.Quote_Endpoint(symbol, datatype='json')
Quote_Endpoint

{'Global Quote': {'01. symbol': 'AAPL',
  '02. open': '161.5300',
  '03. high': '162.4700',
  '04. low': '161.2710',
  '05. price': '162.3600',
  '06. volume': '46805370',
  '07. latest trading day': '2023-03-30',
  '08. previous close': '160.7700',
  '09. change': '1.5900',
  '10. change percent': '0.9890%'}}

In [32]:
Quote_Endpoint = av.Quote_Endpoint(symbol, datatype='csv')
Quote_Endpoint

,symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
0,AAPL,161.53,162.47,161.271,162.36,46805370,2023-03-30,160.77,1.59,0.9890%


In [33]:
Search_Endpoint = av.Search_Endpoint(keywords = 'microsoft', datatype='json')
Search_Endpoint

{'bestMatches': [{'1. symbol': 'MSF0.FRK',
   '2. name': 'MICROSOFT CORP. CDR',
   '3. type': 'Equity',
   '4. region': 'Frankfurt',
   '5. marketOpen': '08:00',
   '6. marketClose': '20:00',
   '7. timezone': 'UTC+02',
   '8. currency': 'EUR',
   '9. matchScore': '0.6429'},
  {'1. symbol': 'MSFT',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '0.6154'},
  {'1. symbol': '0QYP.LON',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'United Kingdom',
   '5. marketOpen': '08:00',
   '6. marketClose': '16:30',
   '7. timezone': 'UTC+01',
   '8. currency': 'USD',
   '9. matchScore': '0.6000'},
  {'1. symbol': 'MSF.DEX',
   '2. name': 'Microsoft Corporation',
   '3. type': 'Equity',
   '4. region': 'XETRA',
   '5. marketOpen': '08:00',
   '6. marketClose': '20:00',
   '7. timezone': '

In [34]:
Search_Endpoint = av.Search_Endpoint(keywords = 'microsoft', datatype='csv')
Search_Endpoint

,symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore
0,MSF0.FRK,MICROSOFT CORP. CDR,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.6429
1,MSFT,Microsoft Corporation,Equity,United States,09:30,16:00,UTC-04,USD,0.6154
2,0QYP.LON,Microsoft Corporation,Equity,United Kingdom,08:00,16:30,UTC+01,USD,0.6000
3,MSF.DEX,Microsoft Corporation,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.6000
4,MSF.FRK,Microsoft Corporation,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.6000
5,MSFT34.SAO,Microsoft Corporation,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.6000


In [35]:
Market_Status = av.Market_Status()
Market_Status

{'endpoint': 'Global Market Open & Close Status',
 'markets': [{'market_type': 'Equity',
   'region': 'United States',
   'primary_exchanges': 'NASDAQ, NYSE, AMEX, BATS',
   'local_open': '09:30',
   'local_close': '16:00',
   'current_status': 'closed',
   'notes': ''},
  {'market_type': 'Equity',
   'region': 'Canada',
   'primary_exchanges': 'Toronto, Toronto Ventures',
   'local_open': '09:30',
   'local_close': '16:00',
   'current_status': 'closed',
   'notes': ''},
  {'market_type': 'Equity',
   'region': 'United Kingdom',
   'primary_exchanges': 'London',
   'local_open': '08:00',
   'local_close': '16:30',
   'current_status': 'closed',
   'notes': ''},
  {'market_type': 'Equity',
   'region': 'Germany',
   'primary_exchanges': 'XETRA, Berlin, Frankfurt, Munich, Stuttgart',
   'local_open': '08:00',
   'local_close': '20:00',
   'current_status': 'closed',
   'notes': ''},
  {'market_type': 'Equity',
   'region': 'France',
   'primary_exchanges': 'Paris',
   'local_open': '09:

In [40]:
Market_News_Sentiment = av.Market_News_Sentiment(tickers='COIN', topics = 'mergers_and_acquisitions', time_from = '20230301', time_to = '20230330', sort='LATEST', limit='50')
Market_News_Sentiment

{'Information': 'Invalid inputs. Please refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}

In [41]:
Real_GDP = av.Real_GDP(interval='annual', datatype='json')
Real_GDP

{'name': 'Real Gross Domestic Product',
 'interval': 'annual',
 'unit': 'billions of dollars',
 'data': [{'date': '2022-01-01', 'value': '20014.128'},
  {'date': '2021-01-01', 'value': '19609.812'},
  {'date': '2020-01-01', 'value': '18509.143'},
  {'date': '2019-01-01', 'value': '19036.052'},
  {'date': '2018-01-01', 'value': '18609.078'},
  {'date': '2017-01-01', 'value': '18076.651'},
  {'date': '2016-01-01', 'value': '17680.274'},
  {'date': '2015-01-01', 'value': '17390.295'},
  {'date': '2014-01-01', 'value': '16932.051'},
  {'date': '2013-01-01', 'value': '16553.348'},
  {'date': '2012-01-01', 'value': '16253.97'},
  {'date': '2011-01-01', 'value': '15891.534'},
  {'date': '2010-01-01', 'value': '15648.991'},
  {'date': '2009-01-01', 'value': '15236.262'},
  {'date': '2008-01-01', 'value': '15642.962'},
  {'date': '2007-01-01', 'value': '15623.871'},
  {'date': '2006-01-01', 'value': '15315.943'},
  {'date': '2005-01-01', 'value': '14901.269'},
  {'date': '2004-01-01', 'value': 

In [42]:
Real_GDP = av.Real_GDP(interval='annual', datatype='csv')
Real_GDP

,timestamp,value
0,2022-01-01,20014.128
1,2021-01-01,19609.812
2,2020-01-01,18509.143
3,2019-01-01,19036.052
4,2018-01-01,18609.078
...,...,...
89,1933-01-01,817.824
90,1932-01-01,828.061
91,1931-01-01,950.686
92,1930-01-01,1015.751


In [44]:
Real_GDP_per_Capita = av.Real_GDP_per_Capita(datatype='json')
Real_GDP_per_Capita

{'name': 'Real Gross Domestic Product per Capita',
 'interval': 'quarterly',
 'unit': 'chained 2012 dollars',
 'data': [{'date': '2022-10-01', 'value': '60376.0'},
  {'date': '2022-07-01', 'value': '60080.0'},
  {'date': '2022-04-01', 'value': '59688.0'},
  {'date': '2022-01-01', 'value': '59836.0'},
  {'date': '2021-10-01', 'value': '60117.0'},
  {'date': '2021-07-01', 'value': '59169.0'},
  {'date': '2021-04-01', 'value': '58842.0'},
  {'date': '2021-01-01', 'value': '57882.0'},
  {'date': '2020-10-01', 'value': '56993.0'},
  {'date': '2020-07-01', 'value': '56479.0'},
  {'date': '2020-04-01', 'value': '52393.0'},
  {'date': '2020-01-01', 'value': '57279.0'},
  {'date': '2019-10-01', 'value': '58017.0'},
  {'date': '2019-07-01', 'value': '57845.0'},
  {'date': '2019-04-01', 'value': '57418.0'},
  {'date': '2019-01-01', 'value': '57100.0'},
  {'date': '2018-10-01', 'value': '56851.0'},
  {'date': '2018-07-01', 'value': '56831.0'},
  {'date': '2018-04-01', 'value': '56512.0'},
  {'date

In [45]:
Real_GDP_per_Capita = av.Real_GDP_per_Capita(datatype='csv')
Real_GDP_per_Capita

,timestamp,value
0,2022-10-01,60376.0
1,2022-07-01,60080.0
2,2022-04-01,59688.0
3,2022-01-01,59836.0
4,2021-10-01,60117.0
...,...,...
299,1948-01-01,14326.0
300,1947-10-01,14171.0
301,1947-07-01,14018.0
302,1947-04-01,14111.0


In [46]:
Treasury_Yield = av.Treasury_Yield(interval='monthly', maturity='10year', datatype='json')
Treasury_Yield

{'name': '10-Year Treasury Constant Maturity Rate',
 'interval': 'monthly',
 'unit': 'percent',
 'data': [{'date': '2023-02-01', 'value': '3.75'},
  {'date': '2023-01-01', 'value': '3.53'},
  {'date': '2022-12-01', 'value': '3.62'},
  {'date': '2022-11-01', 'value': '3.89'},
  {'date': '2022-10-01', 'value': '3.98'},
  {'date': '2022-09-01', 'value': '3.52'},
  {'date': '2022-08-01', 'value': '2.90'},
  {'date': '2022-07-01', 'value': '2.90'},
  {'date': '2022-06-01', 'value': '3.14'},
  {'date': '2022-05-01', 'value': '2.90'},
  {'date': '2022-04-01', 'value': '2.75'},
  {'date': '2022-03-01', 'value': '2.13'},
  {'date': '2022-02-01', 'value': '1.93'},
  {'date': '2022-01-01', 'value': '1.76'},
  {'date': '2021-12-01', 'value': '1.47'},
  {'date': '2021-11-01', 'value': '1.56'},
  {'date': '2021-10-01', 'value': '1.58'},
  {'date': '2021-09-01', 'value': '1.37'},
  {'date': '2021-08-01', 'value': '1.28'},
  {'date': '2021-07-01', 'value': '1.32'},
  {'date': '2021-06-01', 'value': '1

In [47]:
Treasury_Yield = av.Treasury_Yield(interval='monthly', maturity='10year', datatype='csv')
Treasury_Yield

,timestamp,value
0,2023-02-01,3.75
1,2023-01-01,3.53
2,2022-12-01,3.62
3,2022-11-01,3.89
4,2022-10-01,3.98
...,...,...
834,1953-08-01,2.95
835,1953-07-01,2.93
836,1953-06-01,3.11
837,1953-05-01,3.05


In [48]:
Federal_Funds_Rate = av.Federal_Funds_Rate(interval='monthly', datatype='json')
Federal_Funds_Rate

{'name': 'Effective Federal Funds Rate',
 'interval': 'monthly',
 'unit': 'percent',
 'data': [{'date': '2023-02-01', 'value': '4.57'},
  {'date': '2023-01-01', 'value': '4.33'},
  {'date': '2022-12-01', 'value': '4.10'},
  {'date': '2022-11-01', 'value': '3.78'},
  {'date': '2022-10-01', 'value': '3.08'},
  {'date': '2022-09-01', 'value': '2.56'},
  {'date': '2022-08-01', 'value': '2.33'},
  {'date': '2022-07-01', 'value': '1.68'},
  {'date': '2022-06-01', 'value': '1.21'},
  {'date': '2022-05-01', 'value': '0.77'},
  {'date': '2022-04-01', 'value': '0.33'},
  {'date': '2022-03-01', 'value': '0.20'},
  {'date': '2022-02-01', 'value': '0.08'},
  {'date': '2022-01-01', 'value': '0.08'},
  {'date': '2021-12-01', 'value': '0.08'},
  {'date': '2021-11-01', 'value': '0.08'},
  {'date': '2021-10-01', 'value': '0.08'},
  {'date': '2021-09-01', 'value': '0.08'},
  {'date': '2021-08-01', 'value': '0.09'},
  {'date': '2021-07-01', 'value': '0.10'},
  {'date': '2021-06-01', 'value': '0.08'},
  {'

In [49]:
Federal_Funds_Rate = av.Federal_Funds_Rate(interval='monthly', datatype='csv')
Federal_Funds_Rate

,timestamp,value
0,2023-02-01,4.57
1,2023-01-01,4.33
2,2022-12-01,4.10
3,2022-11-01,3.78
4,2022-10-01,3.08
...,...,...
819,1954-11-01,0.83
820,1954-10-01,0.85
821,1954-09-01,1.07
822,1954-08-01,1.22


In [50]:
CPI = av.CPI(interval='monthly', datatype='json')
CPI

{'name': 'Consumer Price Index for all Urban Consumers',
 'interval': 'monthly',
 'unit': 'index 1982-1984=100',
 'data': [{'date': '2023-02-01', 'value': '300.840'},
  {'date': '2023-01-01', 'value': '299.170'},
  {'date': '2022-12-01', 'value': '296.797'},
  {'date': '2022-11-01', 'value': '297.711'},
  {'date': '2022-10-01', 'value': '298.012'},
  {'date': '2022-09-01', 'value': '296.808'},
  {'date': '2022-08-01', 'value': '296.171'},
  {'date': '2022-07-01', 'value': '296.276'},
  {'date': '2022-06-01', 'value': '296.311'},
  {'date': '2022-05-01', 'value': '292.296'},
  {'date': '2022-04-01', 'value': '289.109'},
  {'date': '2022-03-01', 'value': '287.504'},
  {'date': '2022-02-01', 'value': '283.716'},
  {'date': '2022-01-01', 'value': '281.148'},
  {'date': '2021-12-01', 'value': '278.802'},
  {'date': '2021-11-01', 'value': '277.948'},
  {'date': '2021-10-01', 'value': '276.589'},
  {'date': '2021-09-01', 'value': '274.310'},
  {'date': '2021-08-01', 'value': '273.567'},
  {'d

In [51]:
CPI = av.CPI(interval='monthly', datatype='csv')
CPI

,timestamp,value
0,2023-02-01,300.840
1,2023-01-01,299.170
2,2022-12-01,296.797
3,2022-11-01,297.711
4,2022-10-01,298.012
...,...,...
1317,1913-05-01,9.700
1318,1913-04-01,9.800
1319,1913-03-01,9.800
1320,1913-02-01,9.800


In [52]:
Inflation = av.Inflation(datatype='json')
Inflation

{'name': 'Inflation - US Consumer Prices',
 'interval': 'annual',
 'unit': 'percent',
 'data': [{'date': '2021-01-01', 'value': '4.69785886363739'},
  {'date': '2020-01-01', 'value': '1.23358439630637'},
  {'date': '2019-01-01', 'value': '1.81221007526015'},
  {'date': '2018-01-01', 'value': '2.44258329692818'},
  {'date': '2017-01-01', 'value': '2.13011000365963'},
  {'date': '2016-01-01', 'value': '1.26158320570537'},
  {'date': '2015-01-01', 'value': '0.118627135552435'},
  {'date': '2014-01-01', 'value': '1.62222297740821'},
  {'date': '2013-01-01', 'value': '1.46483265562714'},
  {'date': '2012-01-01', 'value': '2.06933726526059'},
  {'date': '2011-01-01', 'value': '3.15684156862206'},
  {'date': '2010-01-01', 'value': '1.64004344238989'},
  {'date': '2009-01-01', 'value': '-0.35554626629975'},
  {'date': '2008-01-01', 'value': '3.83910029665101'},
  {'date': '2007-01-01', 'value': '2.85267248150136'},
  {'date': '2006-01-01', 'value': '3.22594410070407'},
  {'date': '2005-01-01',

In [53]:
Inflation = av.Inflation(datatype='csv')
Inflation

,timestamp,value
0,2021-01-01,4.697859
1,2020-01-01,1.233584
2,2019-01-01,1.812210
3,2018-01-01,2.442583
4,2017-01-01,2.130110
...,...,...
57,1964-01-01,1.278912
58,1963-01-01,1.239669
59,1962-01-01,1.198773
60,1961-01-01,1.070724


In [54]:
Retail_Sales = av.Retail_Sales(datatype='json')
Retail_Sales

{'name': 'Advance Retail Sales: Retail Trade',
 'interval': 'monthly',
 'unit': 'millions of dollars',
 'data': [{'date': '2023-02-01', 'value': '527697'},
  {'date': '2023-01-01', 'value': '549448'},
  {'date': '2022-12-01', 'value': '657814'},
  {'date': '2022-11-01', 'value': '609862'},
  {'date': '2022-10-01', 'value': '597349'},
  {'date': '2022-09-01', 'value': '577370'},
  {'date': '2022-08-01', 'value': '613416'},
  {'date': '2022-07-01', 'value': '600746'},
  {'date': '2022-06-01', 'value': '609933'},
  {'date': '2022-05-01', 'value': '614474'},
  {'date': '2022-04-01', 'value': '594426'},
  {'date': '2022-03-01', 'value': '597077'},
  {'date': '2022-02-01', 'value': '506400'},
  {'date': '2022-01-01', 'value': '518146'},
  {'date': '2021-12-01', 'value': '632849'},
  {'date': '2021-11-01', 'value': '579687'},
  {'date': '2021-10-01', 'value': '557737'},
  {'date': '2021-09-01', 'value': '532840'},
  {'date': '2021-08-01', 'value': '553822'},
  {'date': '2021-07-01', 'value': 

In [55]:
Retail_Sales = av.Retail_Sales(datatype='csv')
Retail_Sales

,timestamp,value
0,2023-02-01,527697
1,2023-01-01,549448
2,2022-12-01,657814
3,2022-11-01,609862
4,2022-10-01,597349
...,...,...
369,1992-05-01,152420
370,1992-04-01,147175
371,1992-03-01,142488
372,1992-02-01,131244


In [56]:
Durables = av.Durables(datatype='json')
Durables

{'name': 'Manufacturer New Orders: Durable Goods',
 'interval': 'monthly',
 'unit': 'millions of dollars',
 'data': [{'date': '2023-02-01', 'value': '252707'},
  {'date': '2023-01-01', 'value': '249969'},
  {'date': '2022-12-01', 'value': '296687'},
  {'date': '2022-11-01', 'value': '259606'},
  {'date': '2022-10-01', 'value': '273120'},
  {'date': '2022-09-01', 'value': '289932'},
  {'date': '2022-08-01', 'value': '279323'},
  {'date': '2022-07-01', 'value': '249456'},
  {'date': '2022-06-01', 'value': '294470'},
  {'date': '2022-05-01', 'value': '263971'},
  {'date': '2022-04-01', 'value': '260366'},
  {'date': '2022-03-01', 'value': '294088'},
  {'date': '2022-02-01', 'value': '250105'},
  {'date': '2022-01-01', 'value': '242492'},
  {'date': '2021-12-01', 'value': '267477'},
  {'date': '2021-11-01', 'value': '244327'},
  {'date': '2021-10-01', 'value': '247086'},
  {'date': '2021-09-01', 'value': '260121'},
  {'date': '2021-08-01', 'value': '250971'},
  {'date': '2021-07-01', 'valu

In [57]:
Durables = av.Durables(datatype='csv')
Durables

,timestamp,value
0,2023-02-01,252707
1,2023-01-01,249969
2,2022-12-01,296687
3,2022-11-01,259606
4,2022-10-01,273120
...,...,...
368,1992-06-01,131268
369,1992-05-01,122274
370,1992-04-01,123268
371,1992-03-01,129309


In [58]:
Unemployment = av.Unemployment(datatype='json')
Unemployment

{'name': 'Unemployment Rate',
 'interval': 'monthly',
 'unit': 'percent',
 'data': [{'date': '2023-02-01', 'value': '3.6'},
  {'date': '2023-01-01', 'value': '3.4'},
  {'date': '2022-12-01', 'value': '3.5'},
  {'date': '2022-11-01', 'value': '3.6'},
  {'date': '2022-10-01', 'value': '3.7'},
  {'date': '2022-09-01', 'value': '3.5'},
  {'date': '2022-08-01', 'value': '3.7'},
  {'date': '2022-07-01', 'value': '3.5'},
  {'date': '2022-06-01', 'value': '3.6'},
  {'date': '2022-05-01', 'value': '3.6'},
  {'date': '2022-04-01', 'value': '3.6'},
  {'date': '2022-03-01', 'value': '3.6'},
  {'date': '2022-02-01', 'value': '3.8'},
  {'date': '2022-01-01', 'value': '4.0'},
  {'date': '2021-12-01', 'value': '3.9'},
  {'date': '2021-11-01', 'value': '4.2'},
  {'date': '2021-10-01', 'value': '4.5'},
  {'date': '2021-09-01', 'value': '4.8'},
  {'date': '2021-08-01', 'value': '5.2'},
  {'date': '2021-07-01', 'value': '5.4'},
  {'date': '2021-06-01', 'value': '5.9'},
  {'date': '2021-05-01', 'value': '5

In [59]:
Unemployment = av.Unemployment(datatype='csv')
Unemployment

,timestamp,value
0,2023-02-01,3.6
1,2023-01-01,3.4
2,2022-12-01,3.5
3,2022-11-01,3.6
4,2022-10-01,3.7
...,...,...
897,1948-05-01,3.5
898,1948-04-01,3.9
899,1948-03-01,4.0
900,1948-02-01,3.8


In [60]:
Nonfarm_Payroll = av.Nonfarm_Payroll(datatype='json')
Nonfarm_Payroll

{'name': 'Total Nonfarm Payroll',
 'interval': 'monthly',
 'unit': 'thousands of persons',
 'data': [{'date': '2023-02-01', 'value': '153955'},
  {'date': '2023-01-01', 'value': '152836'},
  {'date': '2022-12-01', 'value': '155344'},
  {'date': '2022-11-01', 'value': '155642'},
  {'date': '2022-10-01', 'value': '155041'},
  {'date': '2022-09-01', 'value': '153809'},
  {'date': '2022-08-01', 'value': '153285'},
  {'date': '2022-07-01', 'value': '152875'},
  {'date': '2022-06-01', 'value': '153217'},
  {'date': '2022-05-01', 'value': '152291'},
  {'date': '2022-04-01', 'value': '151449'},
  {'date': '2022-03-01', 'value': '150411'},
  {'date': '2022-02-01', 'value': '149606'},
  {'date': '2022-01-01', 'value': '147932'},
  {'date': '2021-12-01', 'value': '150740'},
  {'date': '2021-11-01', 'value': '150543'},
  {'date': '2021-10-01', 'value': '149605'},
  {'date': '2021-09-01', 'value': '147917'},
  {'date': '2021-08-01', 'value': '147159'},
  {'date': '2021-07-01', 'value': '146619'},
 

In [61]:
Nonfarm_Payroll = av.Nonfarm_Payroll(datatype='csv')
Nonfarm_Payroll

,timestamp,value
0,2023-02-01,153955
1,2023-01-01,152836
2,2022-12-01,155344
3,2022-11-01,155642
4,2022-10-01,155041
...,...,...
1005,1939-05-01,30145
1006,1939-04-01,29786
1007,1939-03-01,29804
1008,1939-02-01,29394
